In [1]:
import pandas as pd
import zipfile
import numpy as np
import matplotlib.pyplot as plt

import folium

In [2]:
california_df = pd.read_csv('california_df.csv')

california_df.head()

C:\Users\Samsung\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (21,24,25,27,29,36,49,57,67,68) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,case_id,jurisdiction,officer_id,reporting_district,chp_shift,population,county_city_location,special_condition,beat_type,chp_beat_type,...,bicyclist_injured_count,motorcyclist_killed_count,motorcyclist_injured_count,primary_ramp,secondary_ramp,latitude,longitude,collision_date,collision_time,process_date
0,726202,3600.0,8945,064,5,5,3612,0,0,not chp,...,0,0,0,NaN,NaN,NaN,NaN,2020-07-26,02:50:00,2020-09-30
1,5816493,1502.0,00001,NaN,5,6,1502,0,0,not chp,...,0,0,0,NaN,NaN,35.38353,-119.10416,2014-05-23,21:38:00,2014-07-30
2,5816498,5606.0,00001,NaN,5,4,5606,0,0,not chp,...,0,0,0,NaN,NaN,34.35117,-119.06705,2014-01-04,19:26:00,2014-03-03
3,5816553,1942.0,33260,123,5,7,1942,0,0,not chp,...,0,0,0,NaN,NaN,NaN,NaN,2014-05-19,09:00:00,2014-05-31
4,5816557,1948.0,20767,NaN,5,5,1948,0,0,not chp,...,0,0,0,NaN,NaN,34.42196,-118.65565,2014-04-17,07:55:00,2014-05-30


In [15]:
# dataframe with columns that contain information about victims in collisions
california_counties = california_df[['collision_severity', 'county_city_location']].copy()

# 'county_city_location' column contains code: first 2 figures - county code
california_counties['county_city_location'] = california_counties['county_city_location'].apply(str)
california_counties['county_code'] = california_counties['county_city_location'].str[:-2]
california_counties['county_code'] = california_counties.county_code.apply(lambda row: row if len(row) == 2 else '0'+ row)
#california_counties['county_code'] = california_counties.county_code.apply(lambda row: '0'+ row)
california_counties.head()

,collision_severity,county_city_location,county_code
0,property damage only,3612,36
1,property damage only,1502,15
2,other injury,5606,56
3,property damage only,1942,19
4,property damage only,1948,19


In [16]:
# county codes in California
county_codes = {
    '01': 'ALAMEDA',
    '02': 'ALPINE',
    '03': 'AMADOR',
    '04': 'BUTTE',
    '05': 'CALAVERAS',
    '06': 'COLUSA',
    '07': 'CONTRA COSTA',
    '08': 'DEL NORTE',
    '09': 'EL DORADO',
    '10': 'FRESNO',
    '11': 'GLENN',
    '12': 'HUMBOLDT',
    '13': 'IMPERIAL',
    '14': 'INYO',
    '15': 'KERN',
    '16': 'KINGS',
    '17': 'LAKE',
    '18': 'LASSEN',
    '19': 'LOS ANGELES',
    '20': 'MADERA',
    '21': 'MARIN',
    '22': 'MARIPOSA',
    '23': 'MENDOCINO',
    '24': 'MERCED',
    '25': 'MODOC',
    '26': 'MONO',
    '27': 'MONTEREY',
    '28': 'NAPA',
    '29': 'NEVADA',
    '30': 'ORANGE',
    '31': 'PLACER',
    '32': 'PLUMAS',
    '33': 'RIVERSIDE',
    '34': 'SACRAMENTO',
    '35': 'SAN BENITO',
    '36': 'SAN BERNARDINO',
    '37': 'SAN DIEGO',
    '38': 'SAN FRANCISCO',
    '39': 'SAN JOAQUIN',
    '40': 'SAN LUIS OBISPO',
    '41': 'SAN MATEO',
    '42': 'SANTA BARBARA',
    '43': 'SANTA CLARA',
    '44': 'SANTA CRUZ',
    '45': 'SHASTA',
    '46': 'SIERRA',
    '47': 'SISKIYOU',
    '48': 'SOLANO',
    '49': 'SONOMA',
    '50': 'STANISLAUS',
    '51': 'SUTTER',
    '52': 'TEHAMA',
    '53': 'TRINITY',
    '54': 'TULARE',
    '55': 'TUOLUMNE',
    '56': 'VENTURA',
    '57': 'YOLO',
    '58': 'YUBA',
}

In [17]:
california_counties['county_name'] = california_counties['county_code'].map(county_codes)
california_counties.head()

,collision_severity,county_city_location,county_code,county_name
0,property damage only,3612,36,SAN BERNARDINO
1,property damage only,1502,15,KERN
2,other injury,5606,56,VENTURA
3,property damage only,1942,19,LOS ANGELES
4,property damage only,1948,19,LOS ANGELES


In [49]:
#number of fatal accidents per county
counties_acc = california_counties.loc[california_counties['collision_severity'] == 'fatal'].groupby(by = 'county_name').count()['collision_severity']
counties_acc = pd.DataFrame(data = counties_acc).reset_index()
counties_acc['county_name']= counties_acc['county_name'].str.upper().str.title()

#number of accidents with severe injury per county
counties_inj = california_counties.loc[california_counties['collision_severity'] == 'severe injury'].groupby(by = 'county_name').count()['collision_severity']
counties_inj = pd.DataFrame(data = counties_inj).reset_index()
counties_inj['county_name']= counties_inj['county_name'].str.upper().str.title()

In [48]:
counties_acc.sort_values(by = 'collision_severity').head()

,county_name,collision_severity
45,Sierra,10
1,Alpine,16
24,Modoc,19
21,Mariposa,24
25,Mono,29


In [47]:
counties_inj.sort_values(by = 'collision_severity').head()

,county_name,collision_severity
1,Alpine,45
24,Modoc,49
45,Sierra,83
17,Lassen,112
25,Mono,114


In [51]:
# visualize data with choropleth
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/tests"
)
state_geo = f"{url}/us-counties.json"


n = folium.Map(location=[37.335480, -121.893028], zoom_start=6)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=counties_acc,
    columns=["county_name", "collision_severity"],
    key_on="feature.properties.name",
    fill_color="PuRd",
    threshold_scale=[0, 100, 200, 400, 600, 800, 1000, 2000, 5000],
    fill_opacity=0.7,
    line_opacity=0.2,
    #legend_name="Number of severe car accidents per 100 000 of citizens",
).add_to(n)

folium.LayerControl().add_to(n)

n

In [53]:
# visualize data with choropleth
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/tests"
)
state_geo = f"{url}/us-counties.json"


m = folium.Map(location=[37.335480, -121.893028], zoom_start=6)

folium.Choropleth(
    geo_data=state_geo,
    name="choropleth",
    data=counties_acc,
    columns=["county_name", "collision_severity"],
    key_on="feature.properties.name",
    fill_color="YlOrBr",
    threshold_scale=[0, 100, 500, 1000, 4000, 10000,20000],
    fill_opacity=0.7,
    line_opacity=0.2,
    #legend_name="Number of severe car accidents per 100 000 of citizens",
).add_to(m)

folium.LayerControl().add_to(m)

m